<div style="color: white; background-color: #03303e; display: flex; align-items: flex-start; margin-bottom: 1em">
    <figure style="text-align: center; margin: 0 1em 0.5em 0.5em">
    <img src="tskit_logo.svg" style="height: 3em;" />
    <figcaption style="font-size: 0.6em; line-height: 1em">Interactive workbooks</figcaption>
</figure>
    <h1 style="margin: auto">Tskit-based simulation: a quick introduction</h1></div>
This tour uses the <em>tskit</em> <a href="https://tskit.dev/tskit/docs/stable/python-api.html">Python interface</a>. Press "shift-enter" (or click the toolbar "run" button, &#9654;) to run a notebook cell and move to the next one. Repeat this from the top of the workbook to work gradually through the demonstration code.

## Backward-time simulation

An easy way to simulate a tree sequence is to use the backward-time [_msprime_](https://tskit.dev/msprime/docs/stable/intro.html) coalescent simulator. Below is the code to simulate 5 diploid samples from a total population of a million, but you can easily change the number of samples to e.g. 500_000, which will simulate a million DNA sequences of length $1\times10^4$ basepairs. Re-running this should only take a few seconds, even on a slow computer (try it!).

In [ ]:
import msprime

ts = msprime.sim_ancestry(samples=5, ploidy=2, population_size=1_000_000, sequence_length=1e4, recombination_rate=1e-8)  # NB ploidy=2 is the default, so can be omitted

In a notebook, you can `display()` the resulting tree sequence

In [ ]:
display(ts)  # actually, if this is the last line in the cell, you can even omit the `display()` part, and simply use `ts`

By default, the simulator has grouped the sampled haploid genomes ("sample nodes") into diploid [individuals](https://tskit.dev/tskit/docs/stable/glossary.html#nodes-genomes-or-individuals):

In [ ]:
for ind in ts.individuals():
    print(f"Individual {ind.id} has genomes (nodes) with IDs {ind.nodes}")
    if ind.id > 2:
        print("...")
        break

You can perform analysis on this object, such as calculating MRCAs, and general branch-based statistics, including things like PCA:

In [ ]:
from matplotlib import pyplot as plt
pca_output = ts.pca(2, individuals=range(ts.num_individuals))  # PCA with each individual as a point
plt.scatter(*pca_output.factors.T)
plt.xlabel("Principle component 1")
plt.ylabel("Principle component 2");

### Adding mutations to a simulated ancestry

Often you want a tree sequence with added [sites](https://tskit.dev/tskit/docs/stable/data-model.html#site-table) and [mutations](https://tskit.dev/tskit/docs/stable/data-model.html#mutation-table), which creates realised genetic diversity (but perhaps you [don't really need mutations?](https://tskit.dev/tutorials/no_mutations.html)). Adding neutral mutations can easily be done using `msprime.sim_mutations()`. The code below adds single basepair mutations (causing SNVs) with equal probability between A, T, G, and C, but many other mutation models are available (see https://tskit.dev/msprime/docs/stable/mutations.html)

In [ ]:
ts_with_mutations = msprime.sim_mutations(ts, rate=1e-8)
print(f"Average genetic diversity is {ts_with_mutations.diversity()}")

## Forward-time simulation

Forward-time simulation is much more flexible than backward-time simulation, but requires simulating the entire population. We can't just simulate a small number of genomes from a larger population, as we did with _msprime_ above. 

Various forward-time simulation frameworks exist. A commonly-used one is [SLiM](https://messerlab.org/slim/), which has its own built-in simulation language, Eidos. It's not possible to run SLiM within a browser-based notebook, so below we simply load in the results of the following Eidos script (this simulates a population of 75 diploids, or 150 haploid genomes, a far cry from the millions in the backward-time simulation):

```
initialize() {
    initializeTreeSeq(timeUnit="generations");  // time units appropriate for simple WF model
    initializeMutationRate(1e-8);
    initializeMutationType("m1", 0.5, "e", 0.001);
    initializeGenomicElementType("g1", m1, 1.0);
    initializeGenomicElement(g1, 0, 999999);
    initializeRecombinationRate(1e-8);
}
1 early() { sim.addSubpop("p1", 75); }
100 late() { sim.treeSeqOutput("SLiM_output.trees"); }
```

In [ ]:
import tskit
slim_ts = tskit.load("data/SLiM_output.trees")

Another important feature of forward-time simulations can be seen if we plot out the local "gene tree" at the start of the simulated genome:

In [ ]:
node_label_style = (
    ".node > .lab {font-size: 50%}"
    ".leaf > .sym {fill: magenta}"
    ".leaf > .lab {text-anchor: start; transform: rotate(90deg) translate(6px)}"
)

slim_ts.first().draw_svg(
    size=(1000, 400),
    time_scale="log_time",
    y_axis=True,
    symbol_size=3,
    y_ticks=[0, 1, 10, 100],
    node_labels={u: u for u in slim_ts.samples()},
    style = node_label_style,    
)

You can see that the ancestry of the 150 magenta "sample nodes" does not extend back past 100 generations ("ticks"). At 100 generations ago, there are still 3 separate ancestral lineages present (the local tree has "multiple roots"), meaning that the simulation has not fully coalesced. This means we are missing important posts of the ancestry, needed to fully understand the relationships between the sample nodes.

## Recapitation

We can use _msprime_ to simulate the missing ancestry of an existing forward-time simulation, using the clever trick of "recapitation". The easiest way to do this is to use the "pyslim" library:

In [ ]:
import sys
if sys.platform == 'emscripten':  # only needed for jupyterlite to load pyslim
    import micropip
    await micropip.install("pyslim")

import pyslim

recap_ts = pyslim.recapitate(slim_ts, ancestral_Ne=500)

And the resulting tree sequence will have fully coalesced, so that each local tree has a single root. 

In [ ]:
recap_ts.first().draw_svg(
    size=(1000, 400),
    time_scale="log_time",
    y_axis=True,
    symbol_size=3,
    y_ticks=[0, 1, 10, 100, 1000],
    node_labels={u: u for u in recap_ts.samples()},
    style = node_label_style,    
)

Because by definition, neutral mutations don't affect the genealogy, we can add them after the tree sequence (note that by default, `sim_mutations()` removes the mutations generated by the SLiM simulation):

In [ ]:
mutated_recap_ts = msprime.sim_mutations(recap_ts, rate=1e-8)
print("Genetic diversity based on variable sites is", mutated_recap_ts.diversity())

## Natural selection

Incorporating natural selection into simulations is much easier in forward-time rather than backward-time simulation. There is extensive documentation for doing this in SLiM (chapters 9 and 10 of the SLiM manual). However, there is very limited support for approximating the effect of selective sweeps in backward-time in _msprime_ (see [here](https://tskit.dev/msprime/docs/stable/ancestry.html#sec-ancestry-models-selective-sweeps)), which is demonstrated below:

In [ ]:
params = {"sequence_length": 1e6, "population_size": 1e3, "random_seed": 321}

# define hard sweep model
start_freq = 1.0 / (2 * params["population_size"])
end_freq = 1 - start_freq
sweep_model = msprime.SweepGenicSelection(
    position=params["sequence_length"]/2, start_frequency=start_freq, end_frequency=end_freq, s=0.25, dt=1e-6,
)
ts = msprime.sim_ancestry(10, model=[sweep_model, "hudson"], recombination_rate=1e-7, **params)
mutated_ts = msprime.sim_mutations(ts, rate=1e-7)

This simulation should show the classic "diversity dip" around the selected region. Note that the _tskit_ Python interface makes extensive use of the [numpy](https://numpy.org) numerical Python library.

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

windows = np.linspace(0, mutated_ts.sequence_length, 50)
plt.stairs(mutated_ts.diversity(windows=windows), windows, baseline=None);

## Demography and stdpopsim

One of the major influences on a population genomic simulation is the demographic history of the population(s) in question: bottlenecks, expansions, population subdivision, etc. Both _msprime_ and _SLiM_ have ways to specify complex demographies (e.g. see https://tskit.dev/msprime/docs/stable/demography.html).

However, the Standard Library of Population Genetic Simulation Models ([stdpopsim](https://popsim-consortium.github.io/stdpopsim-docs/stable/introduction.html)) contains already-tested demographic models for many species, which can be much more convenient. Here's an example of dog evolution:

In [ ]:
if sys.platform == 'emscripten':  # only needed for jupyterlite to load stdpopsim & demesdraw
    await micropip.install("stdpopsim")
    await micropip.install("demesdraw")

In [ ]:
import stdpopsim

species = stdpopsim.get_species("CanFam")  # Dog and wolf
model = species.get_demographic_model("EarlyWolfAdmixture_6F14")
named_populations={pop.description: pop.name for pop in model.populations}
contig = species.get_contig("chr1", left=1e6, right=11e6, mutation_rate=model.mutation_rate)

samples = {
    named_populations["Basenji"]: 10,
    named_populations["Dingo"]: 5,
    named_populations["Croatian wolf"]: 5,
    named_populations["Golden jackal"]: 5,
}
engine = stdpopsim.get_engine("msprime")  # If you use "slim", you can e.g. add exons under negative selection using a DFE
ts = engine.simulate(model, contig, samples)

The [demesdraw](https://grahamgower.github.io/demesdraw) software can be used to show a pictoral representation of the demographic model

In [ ]:
import demesdraw

demesdraw.tubes(model.model.to_demes(),
    colours={pop_name: f"tab:{c}" for pop_name, c in zip(samples.keys(), ("blue", "orange", "green", "red"))},
    log_time=True,
);

Now the standard analyses can be carried out with the resulting tree sequence

In [ ]:
def plot_pca_by_pop(ts):
    pca_output = ts.pca(2)  # PCA with each haploid genome as a point
    for pop in ts.populations():
        use = ts.samples(population=pop.id)
        if use.any():
            plt.scatter(*pca_output.factors[use, :].T, label=pop.metadata["description"])
    plt.xlabel("PC 1")
    plt.ylabel("PC 2")
    plt.legend()

plot_pca_by_pop(ts)

## Simulating multiple chromosomes (advanced)

For most purposes, different chromosomes from the same individual can be simulated independently. However, in recent time (e.g. the most recent 10 generations), there is linkage disequilibrium (LD) between chromosomes. This can be simulated in SLiM 5, or in _msprime_ by simulating a single long chromosome, broken up into randomly recombining pieces, as described [in the _msprime_ docs](https://tskit.dev/msprime/docs/stable/ancestry.html#multiple-chromosomes), from which the example below is largely taken. This example treats the genome as a set of 1Mb chromosomes, joined together by 1 basepair regions in which random recombinational segregation occurs (i.e. at rate = $\log(2)$). In this case we should use the "DiscreteTimeWrightFisher" model for the generations when LD exists between the chromosomes; after that we switch to the standard ("hudson") coalescent for efficiency.

In [ ]:
r_chrom = 1e-8
r_break = np.log(2)
chrom_lengths = [10e6, 5e6, 1e6]
chrom_start = []
chrom_end = []
pos = 0
for L in chrom_lengths:
    chrom_start.append(pos)
    pos += L
    chrom_end.append(pos)
    pos += 1
chroms = list(zip(chrom_start, chrom_end))
print("Chromosomes span these positions", chroms)
rates = [r_chrom, r_break, r_chrom, r_break, r_chrom]
rate_map = msprime.RateMap(position=np.array(chroms).flatten(), rate=rates)

models = [msprime.DiscreteTimeWrightFisher(duration=20), "hudson"]

# simulate a single long ('joined together') chromosome.
joined_ts = msprime.sim_ancestry(10, population_size=1000, recombination_rate=rate_map, model=models)

Alternatively, we can do this within _stdpopsim_. Multi-chromosome simulations are not yet native to _stdpopsim_ 0.3, but we can simply pass the required _msprime_ details, by making a generic contig of the right length, with the right recombination map. Then as above, we switch to the "hudson" model after 20 generations.

In [ ]:
# This 3 chromosome simulation may take a little time to run

contig = species.get_contig(length=max(rate_map), mutation_rate=model.mutation_rate)
base_recomb_rate = contig.recombination_map.rate[0]
contig.recombination_map = msprime.RateMap(
    position=np.array(chroms).flatten(),
    rate=[base_recomb_rate, r_break, base_recomb_rate, r_break, base_recomb_rate],
)

joined_ts = engine.simulate(
    model,
    contig,
    samples,
    msprime_model="dtwf",
    msprime_change_model=[(20, "hudson")],
)

If each chromosome needs analysing separately, this single "concatenated" chromosome can be separated up into separate tree sequences using
 [`keep_intervals()`](https://tskit.dev/tskit/docs/stable/python-api.html#tskit.TreeSequence.keep_intervals).

In [ ]:
ts_list = [joined_ts.keep_intervals([region], simplify=False).trim() for region in chroms]

for i, ts in enumerate(ts_list):
    print(f"chr{i+1} (length {ts.sequence_length} bp): diversity = {ts.diversity()}")

If separate chromosomes already exist, e.g. from running separate simulations in parallel, they can be pasted back together using  [`concatenate()`](https://tskit.dev/tskit/docs/stable/python-api.html#tskit.TreeSequence.concatenate).

<div class="alert alert-block alert-info"><b>Note:</b> because we removed the 1bp joining regions, the concatenated tree sequence below is of total length 3,000,000 bp, rather than 3,000,002 as in the original <code>joined_ts</code>.</div>

In [ ]:
concat_ts = ts_list[0].concatenate(*ts_list[1:])

# NB: in this case, because we didn't simplify the separate tree sequences, they should all share the same node table,
# so we could preserve internal node identities in the concatenated tree sequence as follows:

node_map = np.arange(ts_list[0].num_nodes)
concat_ts = ts_list[0].concatenate(*ts_list[1:], node_mappings=[node_map, node_map])

Having a single tree sequence is a simple way of performing analysis over all the chromosomes in the genome

In [ ]:
plot_pca_by_pop(concat_ts)

In [ ]:
windows = np.linspace(0, concat_ts.sequence_length, 65)
windowed_diversity = concat_ts.diversity(windows=windows)

# Now figure out some variables for plotting
chrom_starts = np.concatenate(([0.0], np.cumsum(chrom_lengths)[:-1]))
chrom_ends = chrom_starts + chrom_lengths

# local coordinate within its chromosome
region_idx = np.searchsorted(chrom_ends, windows, side='left')
local_coords = windows - chrom_starts[region_idx]

# choose which ticks to label 
label_every = 2  # (avoid overcrowding)
label_indices = np.arange(0, len(windows), label_every)

labels = [f"{local_coords[i]/1e6:g}" for i in label_indices]
tick_positions = windows[label_indices]

plt.figure(figsize=(16, 3))
plt.stairs(windowed_diversity, windows, baseline=None)
plt.xticks(ticks=tick_positions, labels=labels, fontsize=9)
plt.xlabel("Local position (Mb)")

# --- background shading ---
for i, (start, end) in enumerate(zip(chrom_starts, chrom_ends)):
    plt.text((start+end)/2, max(windowed_diversity) / 5, f"Chr{i+1}", ha="center")
    plt.axvspan(
        start, end,
        color='gray',
        alpha=0.15 if i % 2 == 0 else 0.05,   # alternate shade
        zorder=0
    )

## Further information

This notebook can be used interactively, to test the extensive capabilities of _msprime_. Both the _msprime_ and _SLiM_ simulators have extensive documentation, and helpful support communities. See https://tskit.dev/software/msprime.html and https://tskit.dev/software/SLiM.html.